In [ ]:
import pandas as pd
import networkx as nx
from itertools import combinations
from collections import defaultdict
import community as community_louvain
import matplotlib.pyplot as plt
from networkx.algorithms.community import louvain_communities
from networkx.algorithms.community.quality import modularity
import pickle
import pandas as pd
import os
import networkx as nx
from networkx.algorithms.community import greedy_modularity_communities
import community
from collections import Counter
import networkx as nx
import community
from collections import Counter
import pandas as pd
import pickle
import numpy as np

In [ ]:
with open("graph_IOX.pkl", "rb") as f:
    G = pickle.load(f)

In [ ]:
edges_to_keep = [(u, v) for u, v, d in G.edges(data=True) if d.get('weight', 0) >= 2]
G_sub = G.edge_subgraph(edges_to_keep).copy()

In [ ]:
num_nodes = G_sub.number_of_nodes()
num_edges = G_sub.number_of_edges()

print(f"Number of nodes: {num_nodes}")
print(f"Number of edges: {num_edges}")


In [ ]:
partition = community.best_partition(G_sub)

In [ ]:
modularity_value = community.modularity(partition, G_sub)

In [ ]:
modularity_value

In [ ]:
import pandas as pd
df_user_communities = pd.DataFrame(list(partition.items()), columns=['User_ID', 'Community_ID'])
print(df_user_communities.head())


In [ ]:
df_pol_res1 = pd.read_csv('/a/bear.cs.xxx.edu./disk/bear-b/users/xxww/PolitiX/Dataset/Political Alignment Dataset/PAN_Result_1.csv')
df_pol_res2 = pd.read_csv('/a/bear.cs.xxx.edu./disk/bear-b/users/xxww/PolitiX/Dataset/Political Alignment Dataset/PAN_Result_2.csv')
df_pol_res3 = pd.read_csv('/a/bear.cs.xxx.edu./disk/bear-b/users/xxww/PolitiX/Dataset/Political Alignment Dataset/PAN_Result_3.csv')

In [ ]:
df_part = pd.merge(df_user_communities,df_pol_res1)
df_part = pd.merge(df_part,df_pol_res2)
df_part = pd.merge(df_part,df_pol_res3)

In [ ]:
df_part = df_part.drop(columns=['Unnamed: 0'])

In [ ]:
df_part.groupby('Community_ID').count()

In [ ]:
df_part.columns

In [ ]:
community_sizes = df_part['Community_ID'].value_counts()
valid_communities = community_sizes[community_sizes >= 5].index


In [ ]:
valid_communities

In [ ]:
filtered_df = df_part[df_part['Community_ID'].isin(valid_communities)]

In [ ]:
filtered_df.loc[filtered_df.Community_ID == 0].groupby('Label').count()

In [ ]:
for i in (valid_communities):
    print(i)
    print(len(filtered_df.loc[filtered_df.Community_ID == i]))

In [ ]:
for i in (valid_communities):
    print(i)
    print(filtered_df.loc[filtered_df.Community_ID == i].groupby('PR3').count())

In [ ]:
for i in (valid_communities):
    print()
    print(f"Community_ID: {i}, Total_User: {len(filtered_df.loc[filtered_df.Community_ID == i])}")
    print()
    print(filtered_df.loc[filtered_df.Community_ID == i].groupby('PR_Result').count())

In [ ]:
for i in (valid_communities):
    print(f"Community_ID: {i}, Total_User: {len(filtered_df.loc[filtered_df.Community_ID == i])}")
    print()
    print(filtered_df.loc[filtered_df.Community_ID == i].groupby('Label').count())
    print()

In [ ]:
df_part.loc[df_part.Community_ID == 9].loc[df_part.PR3==1].groupby('PR_Result').count()

In [ ]:
df_part.loc[df_part.Community_ID == 9].loc[df_part.PR3==0].groupby('PR_Result').count()

In [ ]:
filtered_df.loc[filtered_df.Community_ID == 1].groupby('Label').count()

In [ ]:
filtered_df.loc[filtered_df.Community_ID == 1,'Group_ID'] = 'RG2'

In [ ]:
filtered_df.loc[filtered_df.Community_ID == 1]

In [ ]:
final_group['Group_ID'] = 'RG1'

In [ ]:
new_final = pd.concat([final_group, filtered_df.loc[filtered_df.Community_ID == 1]])

In [ ]:
final_group

In [ ]:
filtered_df

In [ ]:

print(len(filtered_df.loc[filtered_df.Community_ID == c_id]))

In [ ]:
filtered_df.loc[(filtered_df.Community_ID ==9) & (df_part.PR3==0), 'Group_ID'] = 'RG8'

In [ ]:
new_final = pd.concat([new_final, filtered_df.loc[(filtered_df.Community_ID ==9) & (df_part.PR3==0)]])

In [ ]:
new_final = pd.read_csv('PolitiX_Political_Group_Detection.csv')

In [ ]:
new_final

# Find Controlled Campaings

In [ ]:
df_dup_camp = pd.read_csv('All_Text_Duplicate_Tweet.csv')

In [ ]:
df_narr = pd.read_csv('Generic_OutCome.csv')

In [ ]:
df_dup_camp.groupby('Cluster_ID').count()

In [ ]:
cluster_size = df_dup_camp.groupby('Cluster_ID').count().reset_index()[['Cluster_ID', 'Tweet_ID']].rename(columns = {'Tweet_ID': 'Cluster_Size'})

In [ ]:
df_narr.loc[df_narr.Generic_OutCome == 'generic']

In [ ]:
valid_cluster_size = cluster_size.loc[~cluster_size.Cluster_ID.isin( df_narr.loc[df_narr.Generic_OutCome == 'generic'].Cluster_ID)].reset_index(drop = True)

In [ ]:
valid_cluster_size

In [ ]:
df_dup_valid = df_dup_camp.loc[df_dup_camp.Cluster_ID.isin(valid_cluster_size.Cluster_ID)]

In [ ]:
#new_final.loc[new_final.Group_ID == grp[0]]

In [ ]:
new_final.Group_ID.unique()

In [ ]:
grp = ['RG1', 'RG2', 'RG3', 'RG4', 'RG5', 'NG1', 'RG7', 'RG9', 'DG3','RG10', 'RG11', 'DG1', 'RG6', 'CHE', 'DG2', 'RG8']

In [ ]:
for i in grp:
    member_ids = new_final.loc[new_final.Group_ID == i, 'User_ID']
    temp = df_dup_valid[df_dup_valid.User_ID.isin(member_ids)]
    temp = temp.groupby('Cluster_ID').count().reset_index()[['Cluster_ID', 'Tweet_ID']]
    temp = temp.rename(columns={'Tweet_ID': f'GRP_Member_{i}'})
    valid_cluster_size = pd.merge(valid_cluster_size, temp, on='Cluster_ID', how='left')
    valid_cluster_size = valid_cluster_size.fillna(0)


In [ ]:
temp = df_dup_valid.loc[df_dup_valid.User_ID.isin(new_final.loc[new_final.Group_ID == grp[0]].User_ID)].groupby('Cluster_ID').count().reset_index()[['Cluster_ID', 'Tweet_ID']]

In [ ]:
valid_cluster_size.columns

In [ ]:
group_id = ['GRP_Member_RG1', 'GRP_Member_RG2',
       'GRP_Member_RG3', 'GRP_Member_RG4', 'GRP_Member_RG5', 'GRP_Member_NG1',
       'GRP_Member_RG7', 'GRP_Member_RG9', 'GRP_Member_DG3', 'GRP_Member_RG10',
       'GRP_Member_RG11', 'GRP_Member_DG1', 'GRP_Member_RG6', 'GRP_Member_CHE',
       'GRP_Member_DG2', 'GRP_Member_RG8']

In [ ]:
for i in group_id:
    print(i)
    print(len(valid_cluster_size.loc[valid_cluster_size[i]> valid_cluster_size.Cluster_Size*0.8]))

In [ ]:
valid_cluster_size

In [ ]:
valid_cluster_size.loc[valid_cluster_size.GRP_Member_RG1> valid_cluster_size.Cluster_Size*0.5]

In [ ]:
new_final.groupby('Group_ID').count().User_ID.sum()

In [ ]:
new_final.groupby('Group_ID').count()

In [ ]:
#new_final.loc[new_final.Group_ID == 'RG6'].groupby('PR_Result').count()

In [ ]:
new_final

In [ ]:
df_foreign_sus = pd.read_csv('Final_Foreign.csv')

In [ ]:
mask = df_foreign_sus.isna().any(axis=1)
df_foreign_sus_inverse = df_foreign_sus[mask]

In [ ]:
df_foreign_sus_inverse

In [ ]:
comm_for = pd.merge(new_final, df_foreign_sus_inverse.drop_duplicates('User_ID').iloc[175:])

In [ ]:
comm_for.groupby(['PR_Result', 'Country']).count()

In [ ]:
comm_for.loc[comm_for.Group_ID == 'RG9']

In [ ]:
#comm_for.loc[comm_for.Group_ID == 'RG9']

In [ ]:
comm_for.groupby('Group_ID').count()

In [ ]:
#comm_for.loc[comm_for.Group_ID == 'RG2']

In [ ]:
comm_for.groupby('Country').count()

In [ ]:
comm_for

In [ ]:
new_final

In [ ]:
df_dup_valid

In [ ]:
len(df_dup_camp.loc[df_dup_camp.User_ID.isin(new_final.User_ID)].Cluster_ID.unique())

In [ ]:
comm_for.groupby(['Group_ID', 'Country']).count()

# Hijacked and Adopted IOS and Campaigns

In [ ]:
df_new_our = df_dup_valid.sort_values(['Cluster_ID', 'Tweet_ID'])

In [ ]:
df_new_our

In [ ]:
df_part

In [ ]:
df_new_our = pd.merge(df_new_our, df_part)

In [ ]:
df_new_our.loc[df_new_our.Label == 3].groupby('Cluster_ID').count()

In [ ]:
df_new_our

In [ ]:
df_new_our.columns

In [ ]:

first_of_each_cluster = df_new_our.groupby('Cluster_ID').first()

# Count how many of those have Label == 3
count_label_3 = (first_of_each_cluster['Label'] == 3).sum()

print("Number of clusters whose first tweet has Label == 3:", count_label_3)


In [ ]:

only_label_3_clusters = df_new_our.groupby('Cluster_ID')['Label'].apply(lambda x: (x == 3).all())

count_only_label_3 = only_label_3_clusters.sum()

print("Number of clusters that contain only Label == 3 tweets:", count_only_label_3)


In [ ]:
df_part4 = pd.merge(df_pol_res1, df_pol_res2)
df_part4 = pd.merge(df_part4,df_pol_res3)

In [ ]:
df_part4

In [ ]:
df_foreign = pd.read_csv('Final_Foreign.csv')

In [ ]:
df_foreign2 = pd.merge(df_foreign, df_part4)

In [ ]:
pd.merge(df_foreign, df_part4).groupby('Label').count()

In [ ]:
pd.merge(df_foreign, df_part4).groupby('PR3').count()

In [ ]:
df_foreign2.loc[df_foreign2.PR_Result == 0].groupby('Country').count()

In [ ]:
df_foreign2.loc[df_foreign2.PR_Result == 1].groupby('Country').count()